In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import io
import re
import tqdm
import string

In [ ]:
EMBEDDING_SIZE = 256
WINDOW_SIZE = 3

In [ ]:
# Define the corpus

# corpus = ['In a small village surrounded by mountains, people lived in harmony with nature. The seasons changed gently, bringing new colors and sounds to the valley. In spring, the flowers bloomed, and birds sang joyfully, filling the air with music. Farmers planted crops in neat rows, while children played near the streams, their laughter echoing through the hills. Summer brought warmth, and the fields turned green with life. The village buzzed with activity, as people harvested fruits and vegetables. Autumn arrived with a cool breeze, turning the leaves into shades of gold and red. The village prepared for winter, gathering wood and storing food. Snow covered the land, and the world seemed to slow down. Families gathered around fires, sharing stories of the past and dreaming of the future. Despite the challenges of each season, the villagers remained hopeful and resilient, always finding joy in the simple moments of life.']

def custom_standardization(input_data):
    input_data.lower()
    # Create translation table
    translator = str.maketrans('', '', string.punctuation)

    # Remove punctuation
    return input_data.lower().translate(translator)

f = open("war_and_peace.txt", "r")
input_data = f.read()
data = custom_standardization(input_data)
corpus = [data]
 
# Convert the corpus to a sequence of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
print("After converting our words in the corpus into vector of integers:")
print(len(sequences[0]))

In [ ]:
word_index = tokenizer.word_index
index_word = {v: k for k, v in word_index.items()}


# Define the parameters
vocab_size = len(tokenizer.word_index) + 1
print("vocab size: ", vocab_size)

# Generate the context-target pairs
contexts = []
targets = []
for sequence in sequences:
    for i in range(WINDOW_SIZE, len(sequence) - WINDOW_SIZE):
        context = sequence[i - WINDOW_SIZE:i] +\
            sequence[i + 1:i + WINDOW_SIZE + 1]
        target = sequence[i]
        contexts.append(context)
        targets.append(target)

# Convert the contexts and targets to numpy arrays
X = np.array(contexts)

y = np.zeros((len(X), vocab_size))
for i in range(0, len(targets)):
    y[i][targets[i]] = 1


# Define the CBOW model
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=EMBEDDING_SIZE,
                    input_length=2*WINDOW_SIZE))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
# Fit model
# model.fit(X[:1000], y, epochs=5, verbose=2)

In [ ]:
model.load_weights('cbow_weights.h5')

W1 = model.get_weights()[0]
W2 = model.get_weights()[1]

In [ ]:
# Additional processing with RELU function
"""def relu(x):
    return max(0.0, x)

weights_new = model.get_weights()[0].copy()
threshold = 0.8

for i in range(len(weights_new)):
    for j in range(len(weights_new[i])):
        if relu(weights_new[i][j]) > threshold:
            weights_new[i][j] = 1
        else:
            weights_new[i][j] = 0
            
index = 35
count = 0
for i in weights_new[index]:
    if i == 1.0:
        count += 1

print(count)"""

In [ ]:
# Test predictions
def predict_target(context_words):
    context_sequence = tokenizer.texts_to_sequences([context_words])[0]
    context_sequence = pad_sequences([context_sequence], maxlen=WINDOW_SIZE * 2)
    prediction = model.predict(context_sequence)
    predicted_word_idx = np.argmax(prediction)
    predicted_word = tokenizer.index_word[predicted_word_idx]
    return predicted_word

# Example prediction
sentense = 'rendered the country turbulent and difficult to'
context_words = []

split_sentense = sentense.split(" ")

if (len(split_sentense) - 1) != WINDOW_SIZE * 2:
    print("ERROR: sentense does not have the correct lenght for testinng")
else:
    for w in split_sentense:
        context_words.append(w)
    del context_words[WINDOW_SIZE]

    predicted_word = predict_target(context_words)
    print(f"Predicted word for context {context_words}: {predicted_word}")